In [6]:
import requests as rq
from bs4 import BeautifulSoup as bs
from collections import OrderedDict as od
from threading import Thread
from queuelib import queue
from datetime import datetime
import time
import random
import json
import csv
import traceback as tb
import re

In [9]:
HOST = 'https://www.famicloud.com.tw'

In [10]:
index_url = HOST

# 上層爬蟲

In [196]:
def famicloud_crawler():
    res_index = rq.get(index_url)
    soup_index = bs(res_index.text, 'lxml')
    soup_li = soup_index.li
    
    
    for i in range (1 ,1000):
        try:
            small_branch = soup_li.select('a')[i].text
            pre_small_branch_url = soup_li.select('a')[i]['href']
            small_branch_url = "{}{}".format(index_url, pre_small_branch_url)
            
            print(small_branch_url)
            #交給中層爬蟲
            pass

        except KeyError:       
            sub_branch = soup_li.select('a')[i].text
            #print(sub_branch)

        except IndexError:
            break   

    print("[INFO] famicloud_crawler complete!")

In [197]:
famicloud_crawler()

https://www.famicloud.com.tw/Branchs/Branch/Index/1/563/564
https://www.famicloud.com.tw/Branchs/Branch/Index/1/287/288
https://www.famicloud.com.tw/Branchs/Branch/Index/1/908/914
https://www.famicloud.com.tw/Branchs/Branch/Index/1/565/566
https://www.famicloud.com.tw/Branchs/Branch/Index/312/313/316
https://www.famicloud.com.tw/Branchs/Branch/Index/312/895/896
https://www.famicloud.com.tw/Branchs/Branch/Index/312/895/897
https://www.famicloud.com.tw/Branchs/Branch/Index/312/356/357
https://www.famicloud.com.tw/Branchs/Branch/Index/312/356/358
https://www.famicloud.com.tw/Branchs/Branch/Index/312/548/549
https://www.famicloud.com.tw/Branchs/Branch/Index/312/607/608
https://www.famicloud.com.tw/Branchs/Branch/Index/312/504/505
https://www.famicloud.com.tw/Branchs/Branch/Index/312/504/506
https://www.famicloud.com.tw/Branchs/Branch/Index/312/504/507
https://www.famicloud.com.tw/Branchs/Branch/Index/312/504/508
https://www.famicloud.com.tw/Branchs/Branch/Index/2/9/27
https://www.famicloud

# 下層爬蟲

In [15]:
def product_page_crawler(product_url):
    res = rq.get(product_url)
    soup = bs(res.text, 'lxml')
    
    product_dict = od()
    food_info_dict = od()  
    
    food_name = ''
    
    #找到最後一個有strong標籤的div，就會是內容
    div_index_needed = 0
    for div_index in range(1, 70):
        if hasattr(soup.select("div")[div_index].strong, 'text'):
            if div_index > div_index_needed:
                div_index_needed = div_index
            #print(div_index_needed)
    #print(div_index_needed)
    
    try:
        soup.select("div")[div_index_needed].strong.text
        content = soup.select("div")[div_index_needed]
        new_content = content
        
        while(new_content.strong != None):
            
            try:
                food_name += new_content.strong.text
                new_content.strong.extract()
                
            except AttributeError:
                break
                
        food_info_dict["food_name"] = food_name
        #print(food_info_dict)
        
    #------以下應該都沒用了-----
    except AttributeError:
        soup.select("div")[54].strong.text
        content = soup.select("div")[54]
        new_content = content
        
        while(new_content.strong != None):
            
            try:   
                food_name += new_content.strong.text
                new_content.strong.extract()
                
            except AttributeError:
                break
                
        food_info_dict["food_name"] = food_name
    #------以上應該都沒用了-----    
    
    except:
        print("[***ERROR***]can not find food_name!")
        
    while(True):
        
        try:
            new_content.br.replace_with(' ')
            
        except AttributeError:
            break
   
    food_info_and_other = new_content.text.strip().split() 
    
    #original_price
    food_info_dict["original_price"] = soup.select(".BOriginalPrice")[0].text
    #discount_price
    food_info_dict["discount_price"] = soup.select(".OnSalePrice > b")[0].text
    #stored_method
    food_info_dict["stored_method"] = soup.select(".BProductLabel")[0].img['alt']
    #category
    category_list = []
    a_hrefs = soup.select('#map-list')[0]
    branch_levels = a_hrefs.select('a')
    for branch in branch_levels:
        if(branch.text != '首頁'):
            category_list.append(branch.text)
    food_info_dict["category"] = category_list
    #weight
    food_info_dict["weight"] = food_info_and_other[1]
    #expired_date
    food_info_dict["expired_date"] = food_info_and_other[7]
    #country_of_origin
    food_info_dict["country_of_origin"] = food_info_and_other[5]
    
    #把food_info字典放入product_dict字典
    product_dict["food_info"] = food_info_dict
    
    #steps(未完成)
    #product_dict["steps"] = null
    
    #product_img_url
    product_dict["product_img_url"] = soup.select(".product_img_box")[0].img['src']
    #product_characteristic_img_url
    
    try:
        #不在P標籤內
        product_dict["product_characteristic_img_url"] = soup.select('div > img')[1]['src']
        #rint(product_dict)
        
    except IndexError:
        #在P標籤內
        product_dict["product_characteristic_img_url"] = soup.select("div > p > img")[0]['src']
    
    #web_url
    product_dict["web_url"] = product_url
    
    print(product_dict)

# 多頁測試

In [12]:
product_url_list = [
    'https://www.famicloud.com.tw/Branchs/Branch/Product/3/13/50/3872',
    'https://www.famicloud.com.tw/Branchs/Branch/Product/312/504/505/1170',
    'https://www.famicloud.com.tw/Branchs/Branch/Product/3/14/59/1288',
    'https://www.famicloud.com.tw/Branchs/Branch/Product/2/10/42/6012',
    'https://www.famicloud.com.tw/Branchs/Branch/Product/2/12/49/4258',
    'https://www.famicloud.com.tw/Branchs/Branch/Product/622/745/758/4118',
    'https://www.famicloud.com.tw/Branchs/Branch/Product/622/1278/1279/6289',
    'https://www.famicloud.com.tw/Branchs/Branch/Product/5/233/903/5056',
    'https://www.famicloud.com.tw/Branchs/Branch/Product/4/1055/1056/5286',
    'https://www.famicloud.com.tw/Branchs/Branch/Product/5/1023/1070/7001',
    'https://www.famicloud.com.tw/Branchs/Branch/Product/254/344/614/6688',
    'https://www.famicloud.com.tw/Branchs/Branch/Product/879/1016/1017/6361',
    'https://www.famicloud.com.tw/Branchs/Branch/Product/948/967/968/6951'
]

In [16]:
for product_url in product_url_list:
    product_page_crawler(product_url)

OrderedDict([('food_info', OrderedDict([('food_name', '家香豬五花火鍋肉片'), ('original_price', '原價$168'), ('discount_price', '145'), ('stored_method', '冷凍'), ('category', ['肉品肉類', '豬肉', '肉片\\肉條']), ('weight', '300g±5%/包'), ('expired_date', '180天'), ('country_of_origin', '台灣')])), ('product_img_url', 'https://familycloud.blob.core.windows.net/products/10001653_0.jpg'), ('product_characteristic_img_url', 'https://familycloud.blob.core.windows.net/products/3872/1653-2NEW2.jpg'), ('web_url', 'https://www.famicloud.com.tw/Branchs/Branch/Product/3/13/50/3872')])
OrderedDict([('food_info', OrderedDict([('food_name', '築地一番鮮1855濕式熟成美國安格斯PRIME厚切沙朗牛排'), ('original_price', '原價$600'), ('discount_price', '359'), ('stored_method', '冷凍'), ('category', ['鮮食家推薦', '露營方便吃', '野炊生鮮']), ('weight', '500±30g/片'), ('expired_date', '365天'), ('country_of_origin', '美國')])), ('product_img_url', 'https://familycloud.blob.core.windows.net/products/10000378_0.jpg'), ('product_characteristic_img_url', 'https://familycloud.blob